In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import inputparser

In [4]:
input_vector, y_test = inputparser.make_input()

# x-test, y-test are 225 data points, remove first element for verification
x_verification = []
x_verification.append(input_vector[0])
y_verification = y_test[0]

x_input = input_vector[1:]
y_input = []
for i in y_test[1:]:
    y_input.append(i)

In [5]:
# Random seeding and hidden layer tuning

# Input data
X = torch.tensor(x_input.tolist())
Y = torch.tensor(y_input)

# Split data
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Define the model
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Loss function
criterion = nn.MSELoss()

# Hidden layer tunning
hidden_sizes = list(range(200, 301, 10))

# Random seeding tunning
seed_range = range(1, 50)

val_mses = []
for seed in seed_range:
    for hidden_size in hidden_sizes:

        torch.manual_seed(seed)
        np.random.seed(seed)
        input_size = len(x_input[0])
        model = NeuralNet(input_size, hidden_size)
        optimizer = optim.Adam(model.parameters(), lr=0.01)
        best_val_loss = float('inf')
        patience = 10
        early_stop_counter = 0

        for epoch in range(2000):
            outputs = model(torch.tensor(X_train, dtype=torch.float32))
            loss = criterion(outputs.squeeze(), Y_train)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                val_outputs = model(torch.tensor(X_val, dtype=torch.float32))
                val_loss = criterion(val_outputs.squeeze(), Y_val)

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                early_stop_counter = 0
            else:
                early_stop_counter += 1
                if early_stop_counter >= patience:
                    break
        val_mses.append((seed, hidden_size, val_loss.item()))

# Results
best_seed, best_hidden_size, best_val_mse = min(val_mses, key=lambda x: x[2])
print(f"Optimal Random Seed: {best_seed}, Optimal Hidden Layer Size: {best_hidden_size}, Validation MSE: {best_val_mse}")


Optimal Random Seed: 44, Optimal Hidden Layer Size: 290, Validation MSE: 0.017765888944268227


In [7]:
# Epoch tuning 

# Random seeding
torch.manual_seed(44)
np.random.seed(44)

# Input data
X = torch.tensor(x_input.tolist())
Y = torch.tensor(y_input)

# Split data
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_size = len(x_input[0])
# Optimal hidden layer size
hidden_size = 290  
model = NeuralNet(input_size, hidden_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
best_val_loss = float('inf')
optimal_epoch = None

# Training loop
for epoch in range(4000):

    outputs = model(torch.tensor(X_train, dtype=torch.float32))
    loss = criterion(outputs.squeeze(), Y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        val_outputs = model(torch.tensor(X_val, dtype=torch.float32))
        val_loss = mean_squared_error(val_outputs.squeeze().detach().numpy(), Y_val)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        optimal_epoch = epoch + 1
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/4000], Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}')

# Results
print(f'Optimal Epoch: {optimal_epoch}, Validation MSE: {best_val_loss:.8f}')

Epoch [100/4000], Loss: 0.0056, Validation Loss: 0.0253
Epoch [200/4000], Loss: 0.0028, Validation Loss: 0.0197
Epoch [300/4000], Loss: 0.0022, Validation Loss: 0.0354
Epoch [400/4000], Loss: 0.0019, Validation Loss: 0.0553
Epoch [500/4000], Loss: 0.0017, Validation Loss: 0.0777
Epoch [600/4000], Loss: 0.0015, Validation Loss: 0.1080
Epoch [700/4000], Loss: 0.0013, Validation Loss: 0.0685
Epoch [800/4000], Loss: 0.0011, Validation Loss: 0.0681
Epoch [900/4000], Loss: 0.0009, Validation Loss: 0.0998
Epoch [1000/4000], Loss: 0.0008, Validation Loss: 0.0211
Epoch [1100/4000], Loss: 0.0010, Validation Loss: 0.0230
Epoch [1200/4000], Loss: 0.0006, Validation Loss: 0.0624
Epoch [1300/4000], Loss: 0.0048, Validation Loss: 0.0437
Epoch [1400/4000], Loss: 0.0006, Validation Loss: 0.0425
Epoch [1500/4000], Loss: 0.0058, Validation Loss: 0.0555
Epoch [1600/4000], Loss: 0.0155, Validation Loss: 0.0345
Epoch [1700/4000], Loss: 0.0005, Validation Loss: 0.0706
Epoch [1800/4000], Loss: 0.0005, Validat